# Implementing LSA on News Articles using Sklearn

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Import Libraries

In [0]:
import nltk
from nltk.corpus import stopwords
import re
import string
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
import pandas as pd
 
import numpy as np
 
stemmer = PorterStemmer()

In [0]:
import os
from IPython.display import clear_output

In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Download and Preprocess Data
Download data from drive and preprocess by removing stop words, tokenizing and stemming. Store in panda dataframes

In [0]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [0]:
updated_stopwords = stopwords.words('english')
updated_stopwords.remove('he')
updated_stopwords.remove('him')
updated_stopwords.remove('his')
updated_stopwords.remove('himself')
updated_stopwords.remove('she')
updated_stopwords.remove("she's")
updated_stopwords.remove('her')
updated_stopwords.remove('hers')
updated_stopwords.remove('herself')
updated_stopwords.append('said')
updated_stopwords.append('also')
updated_stopwords.append('would')

In [0]:
row_list = []

def load_data (Newspaper):
  global row_list
  
  female_name1 = "firdous ashiq"
  female_name2 = "marriyum aurangzeb"
  female_name3 = "maryam aurangzeb"
  male_name1 = "fawad chaudhry"
  male_name2 = "ch fawad"
  male_name3 = "fawad hussain"
  

  row_list = []
  for date in os.listdir(base_dir + Newspaper):
    for section in os.listdir(base_dir + Newspaper + '/' + date):
      for article in os.listdir(base_dir + Newspaper + '/' + date + '/' + section):
        if (article[-4:] == '.txt'):
          
          clear_output(wait=True)
          with open(base_dir + Newspaper + '/' + date + '/' + section + '/' +  article, 'r') as handler:
            body = handler.read()
            
            pepsi_body = body.lower() #pepsi_body is body in lower letter, dont get thirstyyy
#             thicc = False
            if((female_name1 in pepsi_body or female_name2 in pepsi_body or female_name3 in pepsi_body) and (male_name1 in pepsi_body or male_name2 in pepsi_body or male_name3 in pepsi_body) ):
#               thicc = not thicc
              body = ' '.join(s for s in body.split() if not any(c.isdigit() for c in s))
              body = body.translate(str.maketrans('','',string.punctuation)).lower()
              body = nltk.word_tokenize(body)
              body = [stemmer.stem(word) for word in body if not word in updated_stopwords]
              body = " ".join(body)
          
              row = {'Date': date, 'Section': section, 'Newspaper': Newspaper, 'Body': body, "Gender": "MaleAndFemale"}
              row_list.append(row)
              print(len(row_list))
             
            elif(female_name1 in pepsi_body or female_name2 in pepsi_body or female_name3 in pepsi_body):
#               thicc = not thicc
              body = ' '.join(s for s in body.split() if not any(c.isdigit() for c in s))
              body = body.translate(str.maketrans('','',string.punctuation)).lower()
              body = nltk.word_tokenize(body)
              body = [stemmer.stem(word) for word in body if not word in updated_stopwords]
              body = " ".join(body)
          
              row = {'Date': date, 'Section': section, 'Newspaper': Newspaper, 'Body': body, "Gender": "Female"}
              row_list.append(row)
              print(len(row_list))
              
            elif(male_name1 in pepsi_body or male_name2 in pepsi_body or male_name3 in pepsi_body):
#               thicc = not thicc
              body = ' '.join(s for s in body.split() if not any(c.isdigit() for c in s))
              body = body.translate(str.maketrans('','',string.punctuation)).lower()
              body = nltk.word_tokenize(body)
              body = [stemmer.stem(word) for word in body if not word in updated_stopwords]
              body = " ".join(body)
          
              row = {'Date': date, 'Section': section, 'Newspaper': Newspaper, 'Body': body, "Gender": "Male"}
              row_list.append(row)
              print(len(row_list))

In [0]:
load_data('The_News_International')

498


In [0]:
List_The_News_International = row_list
df_The_News_International = pd.DataFrame(List_The_News_International)
print('Total Rows: ', len(df_The_News_International.index))
df_The_News_International.tail()

Total Rows:  314


,Body,Date,Gender,Newspaper,Section
309,asad ’ resign show govt ’ failur kaira lalamus...,2019_04_19,Male,The_News_International,3_National News
310,posit tumbl cabinet quak islamabad major devel...,2019_04_19,MaleAndFemale,The_News_International,8_National News
311,cabinet split amnesti crackdown islamabad fede...,2019_04_18,Male,The_News_International,1_Front Page
312,cabinet split amnesti crackdown islamabad fede...,2019_04_18,Male,The_News_International,8_National News
313,punjab bureaucrat reshuffl sever chang edu pol...,2019_04_18,Male,The_News_International,8_National News


In [0]:
df_The_News_International['Gender'=='Male'].count()
df_The_News_International['Gender'=='MaleAndFemale'].count()
df_The_News_International['Gender'=='Female'].count()

NameError: ignored

In [0]:
print(df_The_News_International.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 5 columns):
Body         314 non-null object
Date         314 non-null object
Gender       314 non-null object
Newspaper    314 non-null object
Section      314 non-null object
dtypes: object(5)
memory usage: 12.3+ KB
None


In [0]:
df_The_News_International['Date'] =  pd.to_datetime(df_The_News_International['Date'], format='%Y_%m_%d')

In [0]:
import re

def uniform_sections(name):
  pattern = '1|2|3|4|5|6|7|8|9|0|_| |Page|News'
  result = re.sub(pattern, '', name)

  return result

In [0]:
df_The_News_International['Section'] = df_The_News_International['Section'].apply(lambda x: uniform_sections(x))
df_The_News_International.head()

,Body,Date,Gender,Newspaper,Section
0,pmln mpa ’ meet pm deni confirm lahor pakistan...,2019-07-01,Male,The_News_International,Front
1,commiss set crack benami islamabad extend asse...,2019-07-01,Female,The_News_International,Front
2,firdou condemn opposit ’ demand fresh poll isl...,2019-07-01,Female,The_News_International,National
3,pmln mpa ’ meet pm deni confirm lahor pakistan...,2019-07-01,Male,The_News_International,National
4,commiss set crack benami islamabad extend asse...,2019-07-01,Female,The_News_International,National


## Transform with TF-IDF

In [0]:
transformer = TfidfVectorizer()
tfidf = transformer.fit_transform(df_The_News_International.Body)
# print(transformer)

In [0]:
transformer.get_feature_names()

['aamer',
 'aamir',
 'aap',
 'aasia',
 'abad',
 'abadi',
 'abandon',
 'abat',
 'abba',
 'abbasi',
 'abbasil',
 'abbasith',
 'abbrevi',
 'abdelqadar',
 'abdic',
 'abdu',
 'abdul',
 'abet',
 'abettor',
 'abid',
 'abil',
 'abject',
 'abl',
 'abli',
 'abolit',
 'aboutit',
 'abovement',
 'abreast',
 'abroad',
 'abroadit',
 'abroadth',
 'abrupt',
 'abruptli',
 'abruptlydur',
 'abscond',
 'absenc',
 'absolut',
 'abstract',
 'abu',
 'abus',
 'abuzz',
 'academ',
 'academi',
 'acceler',
 'accept',
 'access',
 'accid',
 'accident',
 'accommod',
 'accompani',
 'accomplish',
 'accord',
 'accordingli',
 'account',
 'accountabilitytalk',
 'accountabilityto',
 'accountableon',
 'accounth',
 'accru',
 'accumul',
 'accur',
 'accuraci',
 'accuracyhowev',
 'accus',
 'ace',
 'achakzai',
 'achiev',
 'acknowledg',
 'acquir',
 'acquit',
 'acr',
 'across',
 'acrosstheboard',
 'act',
 'action',
 'actionth',
 'activ',
 'activist',
 'actor',
 'actual',
 'acumen',
 'acut',
 'ad',
 'adam',
 'adb',
 'add',
 'addeda'

## Singular Value Decomposition
We now perform the SVD computation to extract the feature representations of the words. The thing to keep in mind is that the TruncatedSVD model returns the components for the rows alone. Therefore, if the rows of the tf-idf matrix represent the documents, then the TruncatedSVD model will return the feature vectors of the documents. To retrieve the feature vectors of the words, we can simply fit the transpose of the tf-idf matrix to the model.

Here we are interested in finding term-term similarity and hence use the transpose of the tf-idf matrix as the input to the SVD model.

In [0]:
svd = TruncatedSVD(n_components = 300)
lsa = svd.fit_transform(tfidf.T)

## Term-Term Similarity
We now explore the LSA model to compute term-term similarity.

We define two functions for this purpose: 
1.   **getClosestTerm** returns the term in the corpus that is most similar to the queried term.

2.   **kClosestTerms** returns k terms that are closest to the queried term.



In [0]:
def getClosestTerm(term,transformer,model):
 
    term = stemmer.stem(term)
    index = transformer.vocabulary_[term]      
 
    model = np.dot(model,model.T)
    searchSpace =np.concatenate( (model[index][:index] , model[index][(index+1):]) )  
 
    out = np.argmax(searchSpace)
 
    if out<index:
        return transformer.get_feature_names()[out]
    else:
        return transformer.get_feature_names()[(out+1)]

In [0]:
def kClosestTerms(k,term,transformer,model):
 
    term = stemmer.stem(term)
    index = transformer.vocabulary_[term]
 
    model = np.dot(model,model.T)
 
    closestTerms = {}
    for i in range(len(model)):
        closestTerms[transformer.get_feature_names()[i]] = model[index][i]
 
    sortedList = sorted(closestTerms , key= lambda l : closestTerms[l])
 
    return sortedList[::-1][0:k]

## Sample Outputs

In [0]:
kClosestTerms(20,"he",transformer,lsa)


['he',
 'minist',
 'govern',
 'hi',
 'leader',
 'prime',
 'opposit',
 'khan',
 'sharif',
 'imran',
 'parti',
 'peopl',
 'shahbaz',
 'pmln',
 'pakistan',
 'nation',
 'member',
 'fawad',
 'countri',
 'him']

In [0]:
kClosestTerms(20,"she",transformer,lsa)


['she',
 'govern',
 'minist',
 'firdou',
 'opposit',
 'prime',
 'budget',
 'her',
 'nation',
 'leader',
 'pakistan',
 'peopl',
 'imran',
 'dr',
 'awan',
 'maryam',
 'pmln',
 'special',
 'assist',
 'khan']

In [0]:
kClosestTerms(20,"her",transformer,lsa)

['she',
 'her',
 'maryam',
 'firdou',
 'nawaz',
 'minist',
 'prime',
 'father',
 'sharif',
 'govern',
 'pmln',
 'pakistan',
 'dr',
 'special',
 'awan',
 'assist',
 'presid',
 'hi',
 'opposit',
 'nation']

In [0]:
kClosestTerms(20,"him",transformer,lsa)


['he',
 'minist',
 'hi',
 'him',
 'govern',
 'prime',
 'leader',
 'she',
 'opposit',
 'sharif',
 'fawad',
 'khan',
 'imran',
 'shahbaz',
 'nab',
 'peopl',
 'parti',
 'pmln',
 'elect',
 'member']

In [0]:
kClosestTerms(20,"woman",transformer,lsa)

['hereditari',
 'woman',
 'convict',
 'she',
 'parliament',
 'talk',
 'polit',
 'dinner',
 'opposit',
 'iftar',
 'nation',
 'event',
 'gather',
 'maryam',
 'hous',
 'firdou',
 'parti',
 'chair',
 'her',
 'sahiba']

In [0]:
kClosestTerms(20,"man",transformer,lsa)


['he',
 'govern',
 'minist',
 'she',
 'buzdar',
 'hi',
 'prime',
 'man',
 'parti',
 'pti',
 'khan',
 'peopl',
 'pakistan',
 'shah',
 'countri',
 'imran',
 'budget',
 'opposit',
 'punjab',
 'polit']

In [0]:
kClosestTerms(20,"politics",transformer,lsa)


['he',
 'polit',
 'govern',
 'she',
 'minist',
 'parti',
 'opposit',
 'maryam',
 'hi',
 'prime',
 'leader',
 'pakistan',
 'imran',
 'nawaz',
 'countri',
 'pmln',
 'firdou',
 'nation',
 'peopl',
 'pti']

## LDAvis ##

> 



In [0]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [0]:

#modules to build the topic extracting models
from gensim import corpora, models
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
import re
import string
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn import metrics

import os
from IPython.display import clear_output

stemmer = PorterStemmer()

pd.set_option("display.max_colwidth", 200)

## whatever

In [0]:
documents = []
for c in row_list:
  documents.append(c['Body'])

In [0]:
#creating the dictionary
dictionary = corpora.Dictionary([" ".join(documents).split()]) 
print('{} different terms in the corpus'.format(len(dictionary)))
#creating the bag of words object
bow_corpus = [dictionary.doc2bow(text.split()) for text in documents]

7756 different terms in the corpus


In [0]:
tfidf_model = models.TfidfModel(bow_corpus) # creating the tf-idf model
tfidf_corpus = tfidf_model[bow_corpus]

In [0]:

total_topics = 50
lda_model_tfidf = models.LdaModel(corpus=tfidf_corpus, id2word=dictionary, num_topics=total_topics,passes=1, random_state=47)
lda_model_bow = models.LdaModel(corpus=bow_corpus, id2word=dictionary, num_topics=total_topics,passes=1, random_state=47)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [0]:
data = pyLDAvis.gensim.prepare(lda_model_bow, bow_corpus, dictionary)
pyLDAvis.save_html(data, 'lda_infominister.html')
minister_LDAvis

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
24    -0.015913  0.023924       1        1  7.387891
22    -0.025273  0.006466       2        1  6.952961
32    -0.028471  0.016562       3        1  6.268748
48     0.006491 -0.010212       4        1  5.162485
37    -0.030980  0.019758       5        1  4.662194
20    -0.008839 -0.017286       6        1  4.414661
11    -0.033379  0.014251       7        1  3.727337
45     0.007090 -0.024556       8        1  3.657787
18    -0.017371 -0.013839       9        1  3.638097
29    -0.022159 -0.010541      10        1  3.410847
1     -0.027087  0.053206      11        1  3.223034
31    -0.008300 -0.006968      12        1  3.095485
26    -0.039369 -0.011905      13        1  3.056326
46    -0.017565 -0.003408      14        1  2.918147
39    -0.030620  0.013140      15        1  2.753871
35     0.003781 -0.043506      16        1  2.436339
3     -0.008722 -0.020922      17        1  2.232342
28     0.013240 -0.028152      18        1  2.131266
41     0.005061 -0.009595      19        1  1.857871
49    -0.015380  0.009638      20        1  1.814575
14    -0.021050  0.010481      21        1  1.801165
33    -0.034435  0.008010      22        1  1.708168
23    -0.018902  0.007444      23        1  1.689074
47     0.017243  0.002429      24        1  1.628053
12     0.005767 -0.022347      25        1  1.465845
38     0.032811 -0.007055      26        1  1.417531
0     -0.003318  0.001349      27        1  1.351239
44     0.021630  0.018728      28        1  1.322709
10    -0.016165 -0.001441      29        1  1.203620
13     0.009466 -0.013008      30        1  1.143081
27    -0.010536  0.002831      31        1  1.106744
21    -0.007508 -0.018959      32        1  0.955010
30    -0.030388 -0.053358      33        1  0.949543
5     -0.014177 -0.022228      34        1  0.898705
43    -0.001859  0.040690      35        1  0.772942
8     -0.005300  0.023861      36        1  0.682821
2      0.000242 -0.003362      37        1  0.668738
4     -0.017676 -0.018088      38        1  0.653624
15     0.022117  0.021090      39        1  0.563755
42     0.004259 -0.004891      40        1  0.523571
34     0.018384  0.032403      41        1  0.495994
19    -0.008931  0.023296      42        1  0.401807
40     0.017260  0.027141      43        1  0.374179
6      0.017132 -0.033051      44        1  0.308040
36     0.018207 -0.031620      45        1  0.305673
25    -0.003214  0.012464      46        1  0.300218
16     0.065051  0.032379      47        1  0.272260
7      0.098965 -0.027803      48        1  0.197088
17     0.086992  0.018424      49        1  0.030748
9      0.051698  0.018134      50        1  0.005795, topic_info=     Category         Freq        Term        Total  loglift  logprob
2928  Default  1710.000000          he  1710.000000  30.0000  30.0000
4191  Default  1716.000000      minist  1716.000000  29.0000  29.0000
7752  Default  1199.000000           ’  1199.000000  28.0000  28.0000
4820  Default   891.000000    pakistan   891.000000  27.0000  27.0000
2751  Default  1216.000000      govern  1216.000000  26.0000  26.0000
6238  Default   776.000000         she   776.000000  25.0000  25.0000
2974  Default   860.000000          hi   860.000000  24.0000  24.0000
5243  Default   968.000000       prime   968.000000  23.0000  23.0000
7753  Default   700.000000           “   700.000000  22.0000  22.0000
7754  Default   693.000000           ”   693.000000  21.0000  21.0000
3764  Default   502.000000      leader   502.000000  20.0000  20.0000
3598  Default   638.000000        khan   638.000000  19.0000  19.0000
4441  Default   592.000000      nation   592.000000  18.0000  18.0000
5076  Default   419.000000        pmln   419.000000  17.0000  17.0000
4882  Default   465.000000       parti   465.000000  16.0000  16.0000
4716  Default   509.000000     opposit   509.000000  15.0000  15.0000
760   Defau

In [0]:
!ls

clusters_small_noaxes.png  kmeans.png  lda_infominister.html  sample_data
gdrive			   lda.html    lda_mna+mpa.html       ward_clusters.png


In [0]:
from google.colab import files

In [0]:
files.download('kmeans.png')